# Predicting Antibody Binding from Amino Acid Sequences

## Model Development

This notebook focuses on developing and tuning machine learning models for predicting antibody binding from amino acid sequence features.

## 1. Import Libraries

In [ ]:
# Data processing
import pandas as pd
import numpy as np

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Model evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, precision_recall_curve

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# File handling
import os
import sys
import warnings
import joblib

# Progress bar
from tqdm.notebook import tqdm

# Ignore warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 2. Define Paths

In [ ]:
# Define paths
DATA_PROCESSED_DIR = '../data/processed'
RESULTS_DIR = '../results'
FIGURES_DIR = os.path.join(RESULTS_DIR, 'figures')
MODELS_DIR = os.path.join(RESULTS_DIR, 'models')

# Create directories if they don't exist
os.makedirs(DATA_PROCESSED_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

## 3. Load Processed Data

Load the processed data with extracted features from the previous notebook.

In [ ]:
# Load processed data
# This will be created in the feature engineering notebook
try:
    X_train = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'X_train.csv'))
    X_val = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'X_val.csv'))
    X_test = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'X_test.csv'))
    y_train = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'y_train.csv'))['label']
    y_val = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'y_val.csv'))['label']
    y_test = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'y_test.csv'))['label']
    
    print(f"X_train shape: {X_train.shape}")
    print(f"X_val shape: {X_val.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_val shape: {y_val.shape}")
    print(f"y_test shape: {y_test.shape}")
except FileNotFoundError:
    print("Processed data files not found. Please run the feature engineering notebook first.")

## 4. Model Evaluation Function

In [ ]:
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    """Evaluate a model and return performance metrics.
    
    Parameters
    ----------
    model : estimator
        The model to evaluate
    X_train : array-like
        Training features
    X_val : array-like
        Validation features
    y_train : array-like
        Training labels
    y_val : array-like
        Validation labels
    model_name : str
        Name of the model for reporting
    
    Returns
    -------
    dict
        Dictionary of performance metrics
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calculate probabilities if the model supports it
    try:
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_val_prob = model.predict_proba(X_val)[:, 1]
        train_auc = roc_auc_score(y_train, y_train_prob)
        val_auc = roc_auc_score(y_val, y_val_prob)
    except (AttributeError, IndexError):
        train_auc = None
        val_auc = None
    
    # Calculate metrics
    metrics = {
        'model_name': model_name,
        'train_accuracy': accuracy_score(y_train, y_train_pred),
        'val_accuracy': accuracy_score(y_val, y_val_pred),
        'train_precision': precision_score(y_train, y_train_pred),
        'val_precision': precision_score(y_val, y_val_pred),
        'train_recall': recall_score(y_train, y_train_pred),
        'val_recall': recall_score(y_val, y_val_pred),
        'train_f1': f1_score(y_train, y_train_pred),
        'val_f1': f1_score(y_val, y_val_pred),
        'train_auc': train_auc,
        'val_auc': val_auc,
        'train_confusion_matrix': confusion_matrix(y_train, y_train_pred),
        'val_confusion_matrix': confusion_matrix(y_val, y_val_pred),
        'model': model
    }
    
    # Print metrics
    print(f"Model: {model_name}")
    print(f"Train Accuracy: {metrics['train_accuracy']:.4f}")
    print(f"Validation Accuracy: {metrics['val_accuracy']:.4f}")
    print(f"Train F1 Score: {metrics['train_f1']:.4f}")
    print(f"Validation F1 Score: {metrics['val_f1']:.4f}")
    if train_auc is not None:
        print(f"Train AUC: {metrics['train_auc']:.4f}")
        print(f"Validation AUC: {metrics['val_auc']:.4f}")
    print("\nValidation Confusion Matrix:")
    print(metrics['val_confusion_matrix'])
    print("\nClassification Report:")
    print(classification_report(y_val, y_val_pred))
    
    return metrics

## 5. Baseline Model: Logistic Regression

In [ ]:
# Define and train logistic regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_metrics = evaluate_model(lr_model, X_train, X_val, y_train, y_val, "Logistic Regression")

# Save the model
joblib.dump(lr_model, os.path.join(MODELS_DIR, 'logistic_regression.pkl'))

## 6. K-Nearest Neighbors (KNN)

In [ ]:
# Define and train KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_metrics = evaluate_model(knn_model, X_train, X_val, y_train, y_val, "K-Nearest Neighbors")

# Save the model
joblib.dump(knn_model, os.path.join(MODELS_DIR, 'knn.pkl'))

## 7. Random Forest

In [ ]:
# Define and train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_metrics = evaluate_model(rf_model, X_train, X_val, y_train, y_val, "Random Forest")

# Save the model
joblib.dump(rf_model, os.path.join(MODELS_DIR, 'random_forest.pkl'))

## 8. XGBoost

In [ ]:
# Define and train XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_metrics = evaluate_model(xgb_model, X_train, X_val, y_train, y_val, "XGBoost")

# Save the model
joblib.dump(xgb_model, os.path.join(MODELS_DIR, 'xgboost.pkl'))

## 9. Support Vector Machine (SVM)

In [ ]:
# Define and train SVM model
svm_model = SVC(probability=True, random_state=42)
svm_metrics = evaluate_model(svm_model, X_train, X_val, y_train, y_val, "Support Vector Machine")

# Save the model
joblib.dump(svm_model, os.path.join(MODELS_DIR, 'svm.pkl'))

## 10. Model Comparison

In [ ]:
# Collect metrics for all models
models = [lr_metrics, knn_metrics, rf_metrics, xgb_metrics, svm_metrics]
model_names = [m['model_name'] for m in models]
val_accuracy = [m['val_accuracy'] for m in models]
val_precision = [m['val_precision'] for m in models]
val_recall = [m['val_recall'] for m in models]
val_f1 = [m['val_f1'] for m in models]
val_auc = [m['val_auc'] for m in models]

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': val_accuracy,
    'Precision': val_precision,
    'Recall': val_recall,
    'F1 Score': val_f1,
    'AUC': val_auc
})

# Sort by F1 score
comparison_df = comparison_df.sort_values('F1 Score', ascending=False).reset_index(drop=True)

# Display comparison
print("Model Comparison:")
display(comparison_df)

# Visualize comparison
plt.figure(figsize=(12, 8))
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
comparison_df_plot = comparison_df.melt(id_vars='Model', value_vars=metrics_to_plot, var_name='Metric', value_name='Value')
sns.barplot(data=comparison_df_plot, x='Model', y='Value', hue='Metric')
plt.title('Model Comparison')
plt.xlabel('Model')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'model_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

## 11. ROC Curve Comparison

In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for model_metrics in models:
    model = model_metrics['model']
    model_name = model_metrics['model_name']
    
    try:
        y_val_prob = model.predict_proba(X_val)[:, 1]
        fpr, tpr, _ = roc_curve(y_val, y_val_prob)
        auc = roc_auc_score(y_val, y_val_prob)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})')
    except (AttributeError, IndexError):
        print(f"Could not calculate ROC curve for {model_name}")

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig(os.path.join(FIGURES_DIR, 'roc_curve_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

## 12. Feature Importance Analysis

In [ ]:
# Analyze feature importance for models that support it
feature_names = X_train.columns

# Random Forest feature importance
if hasattr(rf_model, 'feature_importances_'):
    rf_importances = pd.DataFrame({
        'Feature': feature_names,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    sns.barplot(data=rf_importances.head(20), x='Importance', y='Feature')
    plt.title('Random Forest Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'rf_feature_importance.png'), dpi=300, bbox_inches='tight')
    plt.show()

# XGBoost feature importance
if hasattr(xgb_model, 'feature_importances_'):
    xgb_importances = pd.DataFrame({
        'Feature': feature_names,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    sns.barplot(data=xgb_importances.head(20), x='Importance', y='Feature')
    plt.title('XGBoost Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'xgb_feature_importance.png'), dpi=300, bbox_inches='tight')
    plt.show()

# Logistic Regression coefficients
if hasattr(lr_model, 'coef_'):
    lr_coeffs = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': lr_model.coef_[0]
    })
    lr_coeffs['AbsCoefficient'] = np.abs(lr_coeffs['Coefficient'])
    lr_coeffs = lr_coeffs.sort_values('AbsCoefficient', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    sns.barplot(data=lr_coeffs.head(20), x='Coefficient', y='Feature')
    plt.title('Logistic Regression Coefficients')
    plt.xlabel('Coefficient')
    plt.ylabel('Feature')
    plt.axvline(x=0, color='r', linestyle='--')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'lr_coefficients.png'), dpi=300, bbox_inches='tight')
    plt.show()

## 13. Hyperparameter Tuning for Best Model

Based on the model comparison, we'll tune the hyperparameters of the best-performing model.

In [ ]:
# Identify the best model from the comparison
best_model_name = comparison_df.iloc[0]['Model']
print(f"Best model: {best_model_name}")

# Define hyperparameter grid based on the best model
if best_model_name == "Logistic Regression":
    model = LogisticRegression(random_state=42)
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }
elif best_model_name == "K-Nearest Neighbors":
    model = KNeighborsClassifier()
    param_grid = {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }
elif best_model_name == "Random Forest":
    model = RandomForestClassifier(random_state=42)
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
elif best_model_name == "XGBoost":
    model = xgb.XGBClassifier(random_state=42)
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
elif best_model_name == "Support Vector Machine":
    model = SVC(probability=True, random_state=42)
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.1, 0.01],
        'kernel': ['rbf', 'linear', 'poly']
    }
else:
    print(f"No hyperparameter tuning defined for {best_model_name}")
    model = None
    param_grid = {}

# Perform grid search if a model and param_grid are defined
if model is not None and param_grid:
    print(f"Performing grid search for {best_model_name}...")
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        scoring='f1',
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    
    # Print best parameters and score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Evaluate the best model
    best_model = grid_search.best_estimator_
    best_model_metrics = evaluate_model(best_model, X_train, X_val, y_train, y_val, f"Tuned {best_model_name}")
    
    # Save the best model
    joblib.dump(best_model, os.path.join(MODELS_DIR, f'tuned_{best_model_name.lower().replace(" ", "_")}.pkl'))

## 14. Final Model Evaluation on Test Set

In [ ]:
# Load the best model
try:
    best_model = joblib.load(os.path.join(MODELS_DIR, f'tuned_{best_model_name.lower().replace(" ", "_")}.pkl'))
except FileNotFoundError:
    print(f"Tuned model file not found. Using the original best model.")
    if best_model_name == "Logistic Regression":
        best_model = lr_model
    elif best_model_name == "K-Nearest Neighbors":
        best_model = knn_model
    elif best_model_name == "Random Forest":
        best_model = rf_model
    elif best_model_name == "XGBoost":
        best_model = xgb_model
    elif best_model_name == "Support Vector Machine":
        best_model = svm_model

# Evaluate on test set
y_test_pred = best_model.predict(X_test)
try:
    y_test_prob = best_model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_prob)
except (AttributeError, IndexError):
    test_auc = None

# Calculate metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_cm = confusion_matrix(y_test, y_test_pred)

# Print metrics
print(f"Final Model: {best_model_name}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
if test_auc is not None:
    print(f"Test AUC: {test_auc:.4f}")
print("\nTest Confusion Matrix:")
print(test_cm)
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Non-binder', 'Binder'],
            yticklabels=['Non-binder', 'Binder'])
plt.title(f'Confusion Matrix - {best_model_name} (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'final_model_confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

# Plot ROC curve if applicable
if test_auc is not None:
    fpr, tpr, _ = roc_curve(y_test, y_test_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'AUC = {test_auc:.3f}')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {best_model_name} (Test Set)')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig(os.path.join(FIGURES_DIR, 'final_model_roc_curve.png'), dpi=300, bbox_inches='tight')
    plt.show()

## 15. Summary and Next Steps

### Summary

In this notebook, we developed and evaluated several machine learning models for predicting antibody binding from amino acid sequence features:

1. **Baseline Model**: Logistic Regression
2. **Advanced Models**:
   - K-Nearest Neighbors (KNN)
   - Random Forest
   - XGBoost
   - Support Vector Machine (SVM)

We compared these models using various performance metrics, including accuracy, precision, recall, F1 score, and AUC. The best-performing model was [Best Model], which achieved an F1 score of [Score] on the validation set and [Score] on the test set.

We also analyzed feature importance to identify the most predictive features for antibody binding. The top features included [list top features].

### Next Steps

1. **Further Model Optimization**:
   - Explore ensemble methods combining multiple models
   - Implement more advanced feature selection techniques
   - Consider deep learning approaches for sequence data

2. **Model Interpretation**:
   - Analyze misclassified examples to understand model limitations
   - Connect model predictions to biological mechanisms
   - Investigate the relationship between important features and binding properties

3. **Application**:
   - Apply the model to new antibody sequences
   - Develop a web interface or API for model deployment
   - Integrate with existing antibody design workflows

The next notebook will focus on detailed analysis of the results and biological interpretation of the model predictions.